In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/SPEECH/Code

In [ ]:
!pip install wandb
!pip install jiwer

This is the main part of the code. It makes models by phases

In [1]:
from dataclasses import dataclass

import PreProcessing, PhaseOneModel, PhaseTwoModel, PhaseThreeModel, Evaluating, PhaseFourModel
import torch
import wandb

c:\python38\lib\site-packages\torchaudio\models\decoder\_ctc_decoder.py:62: UserWarning: The built-in flashlight integration is deprecated, and will be removed in future release. Please install flashlight-text. https://pypi.org/project/flashlight-text/ For the detail of CTC decoder migration, please see https://github.com/pytorch/audio/issues/3088.
  warnings.warn(


Building tokens list:  ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '?']
['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', ' ', '?']


In [2]:
wandb.login()
# Sweep configuration

sweep_config = {
    'method': 'random',
    'metric': {'name': 'WER', 'goal': 'minimize'},
    'name': 'PhaseFourModel',
    'parameters': {
        'wandb_init': {'value': True},
        'epochs': {'value': 400},
        'learning_rate': {'max': 0.01, 'min': 0.00001},
        'batch_size': {'value': 64},
        'n_cnn_layers': {'value': 1},  # Fixed value
        'n_rnn_layers': {'value': 1},  # Fixed value
        'rnn_dim': {'value': 64},
        'n_class': {'value': PreProcessing.NUM_CLASSES + 1},
        'n_feats': {'value': 128},
        'stride': {'value': 2},
        'dropout': {'values': [0, 0.1, 0.3]},
        'lm_weight': {'min': 0, 'max': 3},
        },
    }

wandb: Currently logged in as: ronko (rons-team). Use `wandb login --relogin` to force relogin


In [3]:
@dataclass
class Config:
    def __init__(self, wandb_config=None):
        if wandb_config:
            self = wandb.config
        else:
            learning_rate: float = 0.01
            epochs: int = 300
            batch_size: int = 32
            wandb_init: bool = False

            hyperparams = {
                "n_cnn_layers": 1,
                "n_rnn_layers": 1,
                "rnn_dim": 128,
                "n_class": PreProcessing.NUM_CLASSES+1,
                "n_feats": 128,
                "stride": 2,
                "dropout": 0.5,
            }

In [4]:
def create_model(PhaseNumber, config=None):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Using {} device".format(device))
    architecture = PhaseNumber + 'Model'
    # turn on and off wandb logging
    # start a new wandb run to track this script
    config = config

    test_dataloader = None

    # Now you can create a Dataset and DataLoader for your data
    # wavs, txts = PreProcessing.load_data(mode='train', data_path=PreProcessing.DATA_PATH)
    #
    # dataset = PreProcessing.AudioDatasetV3(wavs, txts)
    # train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.batch_size,
    #                                                shuffle=True,
    #                                                collate_fn=lambda x:
    #                                                PreProcessing.process_data(x))
    # wavs, txts = PreProcessing.load_data(mode='test', data_path=PreProcessing.DATA_PATH)
    # test_dataset = PreProcessing.AudioDatasetV3(wavs, txts)
    # test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size,
    #                                               shuffle=False,
    #                                               collate_fn=lambda x:
    #                                               PreProcessing.process_data(x)
    #                                               )

    # model = PhaseThreeModel.PhaseThreeModel(config,
    #                                         n_cnn_layers=config.hyperparams['n_cnn_layers'],
    #                                         n_rnn_layers=config.hyperparams['n_rnn_layers'],
    #                                         rnn_dim=config.hyperparams['rnn_dim'],
    #                                         n_class=config.hyperparams['n_class'],
    #                                         n_feats=config.hyperparams['n_feats'],
    #                                         stride=config.hyperparams['stride'],
    #                                         dropout=config.hyperparams['dropout'],
    #                                         )
    # model = PhaseFourModel.PhaseFourModel(config)
    wavs, txts = PreProcessing.load_data(mode='train', data_path=PreProcessing.DATA_PATH)
    dataset = PreProcessing.AudioDatasetV2(wavs, txts)
    train_dataloader = torch.utils.data.DataLoader(dataset, batch_size=config.batch_size,
                                                   shuffle=True)

    wavs, txts = PreProcessing.load_data(mode='test', data_path=PreProcessing.DATA_PATH)
    test_dataset = PreProcessing.AudioDatasetV2(wavs, txts)
    test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=config.batch_size,
                                                  shuffle=False)
    model = PhaseTwoModel.PhaseTwoModel()

    return model, train_dataloader, test_dataloader, device

In [5]:
# model, train_dataloader, test_dataloader, device = create_model("PhaseTwo",
#                                                                     config=Config(wandb_init=True))

In [6]:
# criterion = torch.nn.CTCLoss(blank=PreProcessing.BLANK_IDX).to(device)
# PhaseThreeModel.train_model_phase_three(model, train_dataloader, criterion, device, test_dataloader, config=Config(
#     wandb_init=True))

In [7]:
def main_train(config=None):
    with wandb.init(config=config):
        config = wandb.config
        # config.extend({'wandb_init': True})
        print(config)
        model, train_dataloader, test_dataloader, device = create_model("PhaseFour",
                                                                        config=config)
        criterion = torch.nn.CTCLoss(blank=PreProcessing.BLANK_IDX).to(device)
        PhaseThreeModel.train_model_phase_three(model, train_dataloader, criterion, device, test_dataloader, config=config)
        torch.save(model.state_dict(), f"PhaseFourModel_{wandb.run.name}.pt")

In [ ]:
def train_sweep():
    with wandb.init() as run:
        # You can call your main training function here, passing the config
        main_train()  # Assuming 'train' is the main training function

# Running the sweep
sweep_id = wandb.sweep(sweep_config, project='speechRecProj')  # Create the sweep
wandb.agent(sweep_id, train_sweep)


Create sweep with ID: 0a6qigdr
Sweep URL: https://wandb.ai/rons-team/speechRecProj/sweeps/0a6qigdr


wandb: Agent Starting Run: znzxjdo1 with config:
wandb: 	batch_size: 64
wandb: 	dropout: 0.1
wandb: 	epochs: 400
wandb: 	learning_rate: 0.008844704238529656
wandb: 	lm_weight: 0.1
wandb: 	n_class: 28
wandb: 	n_cnn_layers: 1
wandb: 	n_feats: 128
wandb: 	n_rnn_layers: 1
wandb: 	rnn_dim: 64
wandb: 	stride: 2
wandb: 	wandb_init: True
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


{'batch_size': 64, 'dropout': 0.1, 'epochs': 400, 'learning_rate': 0.008844704238529656, 'lm_weight': 0.1, 'n_class': 28, 'n_cnn_layers': 1, 'n_feats': 128, 'n_rnn_layers': 1, 'rnn_dim': 64, 'stride': 2, 'wandb_init': True}
Using cpu device


C:\Users\PC\Documents\ACADEMY\6th semester\Speech Processing\SPEECH\Final Project\Code\LSTMCorpus.py:57: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_sequence = torch.tensor(sequence)
  0%|          | 0/400 [00:00<?, ?it/s]

First Input:  torch.Size([1, 128, 1051]) First Label:  tensor([13, 14,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]) First Label Length:  2
Input txt: noaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
 
 
model preds:  torch.Size([1, 131, 28])
Model Output:  vvvvvveaajevvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
Epoch:  0 / 400  ( 0.0 %)
